In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np
import subprocess
subprocess.run('export TF_INTRA_OP_PARALLELISM_THREADS=12', shell=True)

2022-07-22 17:38:25.894933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-22 17:38:25.894979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CompletedProcess(args='export TF_INTRA_OP_PARALLELISM_THREADS=12', returncode=0)

In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i)
#         homo_pairs[idx][2] = np.add(homo_pair[2],j)
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
train_homo_pairs = homo_pairs
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = c_homo
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [ ]:
setting = {'activation':'tanh', 'nn_shape':(480, 480, 480, 480), 'batch_size':480, 'training_steps':200000,\
'learning_rate': 0.0004, 'decay_rate':0.9, 'decay_per_steps':2000, 'save_step':1000, 'drop_rate':0, 'save_path':'./save/model','seed':10}
NN = nn.NN(setting_dict=setting)
NN.train(train_homo_pairs,train_c_homo)
# NN.test(test_homo_pairs, test_c_homo)


2022-07-22 17:38:36.209908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-22 17:38:36.209957: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-22 17:38:36.210005: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Bai-Group): /proc/driver/nvidia/version does not exist
2022-07-22 17:38:36.210366: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


training step:     0, loss:  0.704294086
training step:  1000, loss:  0.099260427
training step:  2000, loss:  0.083408661
training step:  3000, loss:  0.075999275
training step:  4000, loss:  0.083148822
training step:  5000, loss:  0.068800919
training step:  6000, loss:  0.083688356
training step:  7000, loss:  0.080455631
training step:  8000, loss:  0.087241627
training step:  9000, loss:  0.089851886
training step: 10000, loss:  0.117395140
training step: 11000, loss:  0.128052622
training step: 12000, loss:  0.144637018
training step: 13000, loss:  0.155440167
training step: 14000, loss:  0.174449384
training step: 15000, loss:  0.178013071
training step: 16000, loss:  0.181448430
training step: 17000, loss:  0.181282669
training step: 18000, loss:  0.185390413
training step: 19000, loss:  0.185917124
training step: 20000, loss:  0.179700747
training step: 21000, loss:  0.171150118
training step: 22000, loss:  0.164141446
training step: 23000, loss:  0.145023838
training step: 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

print(np.mean((NN.model(homo_pairs)-c_homo)/c_homo))
x = np.linspace(0, 4, 41)
y = np.linspace(0, 4, 41)
X, Y = np.meshgrid(x, y)
Z = NN.model(train_homo_pairs, training=False).numpy().reshape((41,41))
Z1 = c_homo.reshape((41,41))
print(Z.shape)

fix, ax = plt.subplots()
ax.contourf(x,y, Z)
ax.set_title('pred')